# Fine-Tuning
Fine tuning is a form of transfer learning. Fine-tuning updates a model that has already learned general information with new data, to specialize it. Gradually fine-tune different layers of the model, starting with the topmost layers and progressively including lower layers is one of the techniques used traditionally in while fine-tuning the model. However there are challanges to adapt traditional methods. The challanges are:
1. Gathering labelled data
1. Computational Cost
1. Storage 
1. Out-of-distribution data (leads to Catastrophic Forgetting)

**Catastrophic Forgetting**: This happens when a model learns something new but forgets what it learned before. Imagine if you crammed for a history test and did great, but then forgot most of what you learned when you started studying for a math test.

One of the solutions to above challenges is parameter efficient fine-tuning(PEFT). 

__What is PEFT?__

A method of updating a predefined subset of a model's parameters to tailor it to specific tasks, without the need to modify the entire model, thus saving computational resources is called parameter-efficient fine-tuning. There are several PEFT techinques. Some of them are discussed below. Note: In this notebook we will use LoRA. 

## PEFT techinques: 
### Low-Rank Adaptation (LoRA)

Low-Rank Adaptation (LoRA) is a technique used in deep learning to efficiently fine-tune pre-trained models by decomposing their weight matrices into low-rank representations. This method significantly reduces the number of trainable parameters, making the fine-tuning process more efficient while retaining the model's performance.

<img src="./images/image.png" width="550">

Suppose $\Delta W$ is the weight update for an $A \times B$ weight matrix. Then, we can decompose the weight update matrix into two smaller matrices: $\Delta W = W_A W_B$ , where $W_A$ is an an $A \times r$-dimensional matrix, and $W_B$ is an an $r \times B$-dimensional matrix. Here, we keep the original weight $W$ frozen and only train the new matrices $W_A$ and $W_B$. This, in a nutshell, is the LoRA method, which is illustrated in the figure below.

### Choosing the Rank in LoRA

The rank $r$ in LoRA is a hyperparameter that specifies the complexity of the low-rank matrices used for adaptation. Here's a summary of how choosing $r$ affects the model:

- **Smaller $r$**:
  - **Pros**: Fewer parameters to learn, faster training, reduced computational requirements.
  - **Cons**: Lower capacity to capture task-specific information, potentially leading to lower adaptation quality and poorer performance on the new task.

- **Larger $r$**:
  - **Pros**: Higher capacity to capture task-specific information, potentially better performance on the new task.
  - **Cons**: Increased model complexity, more parameters to learn, longer training times, and higher computational requirements.

Choosing the right $r$ involves balancing model complexity and adaptation capacity to avoid underfitting or overfitting. Experimentation with different $r$ values is crucial to finding the optimal balance for the desired task performance.

---
Prior to proposal of LoRA two main parameter efficient finetuning approaches were there: 
1. Adapters 
2. Prefix tuning 

### Adapters: 
The adapter blocks are extra trainable modules inserted into the existing transformer block. Adapter blocks are inserted after both attention and feedforward layers that have a small number of parameters and can be finetuned while keeping the weights of the pretrained model fixed.

  Each adapter block is a bottleneck-style feedforward module that 
  - decreases the dimensionality of the input via a (trainable) linear layer, 
  - applies a non-linearity, and 
  - restores the original dimensionality of the input via a final (trainable) linear layer.

Only the parameters of these adapters are trained, not of the entire model.

<img src="./images/adapter_approach.png" width="400">

### Prefix Tuning

  Another parameter-efficient finetuning alternative is prefix tuning, which keeps the language model’s parameters frozen and only finetunes a few (trainable) token vectors that are added to the beginning of the model’s input sequence. These token vectors can be interchanged for different tasks.

---
### Comparison between LoRA and Adapters
LoRA (Low-Rank Adaptation) and Adapters are both techniques designed to make the fine-tuning of large pre-trained models more efficient, but they are distinct in their implementation and focus.

- **Similarity**: Both LoRA and Adapters are designed to make fine-tuning more parameter-efficient and computationally efficient by minimizing the number of parameters that need to be updated.
- **Difference**: The key difference lies in their implementation:
  - LoRA modifies the internal structure of existing weight matrices by introducing low-rank updates.
  - Adapters add new, small trainable modules to the existing model architecture.

In summary, while LoRA and Adapters share similar goals in improving fine-tuning efficiency, they achieve this through different mechanisms. LoRA can be thought of as an internal adjustment to the weight matrices, whereas Adapters add external components to the model.

Let's use the LoRA to do the PEFT of the foundation model. 

In [ ]:
from peft import LoraConfig, get_peft_model
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset_builder, load_dataset
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

import numpy as np

In [ ]:
# Get the dataset movie review dataset from the rotten tomatoes

# First let's get the general infomration about the dataset
ds_builder = load_dataset_builder("cornell-movie-review-data/rotten_tomatoes")

# Inspect the dataset information
ds_builder.info.description

In [ ]:
# Insepect the dataset splits
ds_builder.info.splits

In [ ]:
# Inspect the features of the dataset
ds_builder.info.features

In [ ]:
# Get the list of dataset's split name
from datasets import get_dataset_split_names
split_names = get_dataset_split_names("cornell-movie-review-data/rotten_tomatoes")
print("Available splits:", split_names)

In [ ]:
# Le'ts load the actual dataset
# We need all train, validation and test splits so
dataset = load_dataset("cornell-movie-review-data/rotten_tomatoes")

In [ ]:
dataset

In [ ]:
# Get the first row in the dataset
dataset['train'][0]

In [ ]:
# Get the last row in the dataset
dataset['train'][-1]

## Preprocess the dataset
Pre-process the dataset by converting all the text inot tokens for our models

In [ ]:
# Get the tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [ ]:
# Let's see the eos token
tokenizer.eos_token

In [ ]:
# Let's see the pad token
# GPT2 does not have a pad token by default, so we will set it to the eos token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
# Let's see the pad token
print("Pad token:", tokenizer.pad_token)

In [ ]:
tokenized_dataset = {}

for split in split_names:
    # Tokenize the text data
    tokenized_dataset[split] = dataset[split].map(
        lambda examples: tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128),
        batched=True
    )

In [ ]:
tokenized_dataset

Warning from the hugging face:
Model can accept multiple label arguments but none of them should be named "label". This is applicable if we use `label_names` as argument in the `TrainingArguments`. So, we will change the "label" from our dataset to "labels".  

In [ ]:
# Rename 'label' to 'labels' across all splits using a dictionary comprehension
tokenized_dataset = {split: ds.rename_column("label", "labels") for split, ds in tokenized_dataset.items()}

In [ ]:
tokenized_dataset

In [ ]:
# Let's get the label2id and id2label mappings
label2id = {label: i for i, label in enumerate(dataset['train'].features['label'].names)}
id2label = {i: label for label, i in label2id.items()}
print("Label to ID mapping:", label2id)
print("ID to Label mapping:", id2label)

In [ ]:
import json

# After training, save id2label with your model
output_path = "/content/drive/MyDrive/NepGPT/PEFT/lora_adapters"  # or full_merged_model
with open(f"{output_path}/id2label.json", "w") as f:
    json.dump(id2label, f)

## Get the model for sequence classification

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("gpt2",
                                            num_labels=2,  # Binary Classification
                                             device_map="auto", # Automatically map to available device
                                             id2label=id2label,
                                             label2id=label2id)
model.config.pad_token_id = tokenizer.pad_token_id  # Set the pad token id in the model config

You might get this warning:
`Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.`
What is this warning? GPT2 model is used for generating the text not for the classification. But we are using the `gpt2` for classification. So we have score layer in the below architecture which is randomly initialized that acts as the classification head.
```python
GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)
```
When we fin-tune this model using LoRA from the peft library we only choose to fine-tune the layers as shown in the config below:
```python
lora_config = LoraConfig(
    r=8,                    # Rank of the adaptation
    lora_alpha=32,          # Scaling factor
    target_modules=["c_attn", "c_proj"],  # layers to fine-tune in GPT-2
    lora_dropout=0.1,       # Dropout for regularization
    fan_in_fan_out=True,    # Match Conv1D expectation, eliminates the warning
    task_type="SEQ_CLS"   # For sequence Classification
)
```
Since `score.weight` isn’t present in the gpt2 checkpoint, Hugging Face initializes it randomly when you load `AutoModelForSequenceClassification`. This is what triggers the warning.

How to avoid this warning?
We can completely ignore this warning as it is seen just while loading the base model before it is fine-tuned and warning the user you might have to TRAIN this model.

Also we need to keep in mind that when we initialize the model with `peft_model = PeftModel.from_pretrained(base_model, lora_config)`, the score layer remains a regular Linear layer (not adapted by LoRA).

During training, the optimizer (controlled by Trainer) updates `score.weight` fully because unless not frozen or adapted by LoRA. We can check wether the it is optimized or not by
```python
# After initializing peft_model
print("Is score.weight trainable?", peft_model.score.weight.requires_grad)  # Should print True
```
Therefore, later when you load the adapter after fine-tuning with the base model the completely fine-tuned `score.weight` get's loaded. No need to worry about the warning at all.  

Also to be extra cautious we can check after training and after saving the model:
```python
# After training
trainer.train()
post_training_score = peft_model.score.weight[0][:5].tolist()
print("Post-training score.weight:", post_training_score)
peft_model.save_pretrained(output_path)
tokenizer.save_pretrained(output_path)

# Reload
base_model = AutoModelForSequenceClassification.from_pretrained("gpt2")
peft_model = PeftModel.from_pretrained(base_model, output_path)
loaded_score = peft_model.score.weight[0][:5].tolist()
print("Loaded score.weight:", loaded_score)
print("Match?", post_training_score == loaded_score)
```
If `Match? True`, warning is just noise, and model is working as intended.

In [ ]:
model

- `c_attn`: The attention layer’s combined query/key/value projection.
- `c_proj`: The attention output projection.
- `c_fc`: The feed-forward network’s first projection (expanding the hidden size).
- `c_proj (in MLP)`: The feed-forward network’s second projection (reducing back to the hidden size).
- `score`: Output head for the binary classification

In [ ]:
def print_model_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total Number of Parameters: {total_params:,}")
    actual_total_params = (
        total_params - sum(
            p.numel() for p in model.score.parameters()
        )
    )

    total_size_bytes = total_params * 4
    total_size_mb = total_size_bytes / (1024 * 1024)

    print(f"Actual Number of trainable parameters "
          f"considering weight tying: {actual_total_params:,}\n"
          f"Total size of the model is: {total_size_mb:.2f} MB")

In [ ]:
print_model_parameters(model)

In [ ]:
# Let's define the compute metrics function
# Compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = (predictions == labels).mean()
    return {"accuracy": accuracy}

In [ ]:
training_args = TrainingArguments(
    # Checkpoint Saving Settings
    output_dir="./gpt2-lora-rotten-tomatoes-classification/",
    overwrite_output_dir=True,  # Overwrites the output directory if it already exists, avoiding conflicts.

    # Dataset labels
    label_names = ['labels'],

    # Training Epoch and Batch Settings
    num_train_epochs=2,  # Runs training for 3 full passes over the dataset.
    per_device_train_batch_size=8,  # Uses a batch size of 8 per device (e.g., GPU/CPU) during training.
    per_device_eval_batch_size=8,  # Sets evaluation batch size to 8, matching training for consistency.

    # Evaluation Configuration
    eval_strategy="steps",  # Evaluates the model at regular step intervals rather than epoch ends.
    eval_steps=100,  # Performs evaluation every 100 training steps.
    eval_delay=100,  # Delays the first evaluation until after 100 steps, allowing initial training to stabilize.


    # Model Checkpoint Saving Options
    save_strategy="steps",  # Saves checkpoints based on steps, aligning with evaluation strategy.
    save_steps=100,  # Saves a checkpoint every 100 steps, syncing with evaluation for easy rollback.
    save_safetensors=True,  # Uses safetensors format for safer, faster, and more efficient model storage.
    save_total_limit=2,  # Keeps only the 2 most recent checkpoints to manage disk space.

    # Optimizer Hyperparameters
    learning_rate=2e-4,  # Sets a moderate learning rate (0.0002) suitable for fine-tuning tasks like LoRA.
    weight_decay=0.01,  # Applies a small weight decay to prevent overfitting by regularizing weights.

    # Logging Configuration
    logging_dir="./logs",  # Stores training logs in a dedicated directory for analysis.
    logging_steps=10,  # Logs metrics every 10 steps, providing frequent updates on training progress.
    logging_strategy="steps",  # Logs based on step intervals, consistent with evaluation and saving.

    # Model Selection and Reporting
    load_best_model_at_end=True,  # Loads the best-performing model (based on metric) at the end of training.
    metric_for_best_model="accuracy",  # Uses accuracy as the criterion to determine the "best" model.
    report_to="none",  # Disables external reporting (e.g., TensorBoard, WandB) for simplicity or local use.
)

The `Trainer` expects the metric name specified in `metric_for_best_model` (i.e., "accuracy") to exactly match a key in the dictionary returned by `compute_metrics` function.
If "accuracy" isn’t found in the returned dictionary (e.g., you return `{"acc": value}`instead), the Trainer won’t know what value to use to determine the "best" model. This could lead to an error.
The `load_best_model_at_end` feature will not work as expected, because it can’t compare models based on a missing metric.

In [ ]:
# Let's create the trainer
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# How accurate the model is before fine-tuning?

In [ ]:
trainer.evaluate(tokenized_dataset['test'])

## PEFT Model for Sequence Classification using LoRA

In [ ]:
# Let's check the availabe task types from the LoraConfig
from peft import TaskType
print("Available Task Types in LoraConfig:")
for task in TaskType:
    print(f"- {task.name}: {task.value}")

In [ ]:
# Load configuration for lora
lora_config = LoraConfig(
    r=8,                    # Rank of the adaptation
    lora_alpha=32,          # Scaling factor
    target_modules=["c_attn", "c_proj"],  # layers to fine-tune in GPT-2
    lora_dropout=0.1,       # Dropout for regularization
    fan_in_fan_out=True,    # Match Conv1D expectation, eliminates the warning
    task_type="SEQ_CLS"   # For sequence Classification
)

In [ ]:
# Convert the transfomer model to a PEFT model
peft_model = get_peft_model(model, lora_config)

In [ ]:
peft_model

In [ ]:
# After initializing peft_model
print("Is score.weight trainable?", peft_model.score.weight.requires_grad)  # Should print True

### Before PEFT: The Standard GPT-2 Model
The "Before PEFT" version is a straightforward GPT-2 model designed for sequence classification. It starts with a transformer backbone called `GPT2Model`. Inside this, you’ve got:
- **Embeddings**: One layer for word tokens (mapping 50,257 tokens to 768 dimensions) and another for positions (mapping 1,024 positions to 768 dimensions).
- **Dropout**: A dropout layer with a 10% probability to prevent overfitting.
- **Transformer Blocks**: 12 identical blocks stacked together. Each block has:
  - Two **LayerNorm** layers to stabilize training (normalizing 768-dimensional vectors).
  - An **attention mechanism** with two key parts: `c_attn` (a 1D convolution transforming 768 dimensions to 2,304) and `c_proj` (another convolution bringing it back to 768), both with dropout layers (10%) for regularization.
  - A **multi-layer perceptron (MLP)** with `c_fc` (convolution from 768 to 3,072 dimensions), a GELU activation function, and `c_proj` (convolution back from 3,072 to 768), plus dropout.
- **Final LayerNorm**: Another normalization layer at the end of the transformer.
- **Score Layer**: A simple linear layer that takes the 768-dimensional output and maps it to 2 classes (e.g., positive/negative), without bias.

In this setup, every single parameter—about 124 million for GPT-2—is trainable. When you fine-tune this model, you’re adjusting all those weights, which takes a lot of memory and computation.

---

### After PEFT: The LoRA-Enhanced Model
The "After PEFT" version builds on the same GPT-2 foundation but introduces **Parameter-Efficient Fine-Tuning (PEFT)** using **LoRA (Low-Rank Adaptation)**. Here’s how it changes:

- **Top-Level Wrapper**: Instead of just `AutoModelForSequenceClassification`, it’s now a `PeftModelForSequenceClassification` with a `LoraModel` inside. This is the PEFT framework managing the efficiency tweaks.
- **Core Structure**: The base `GPT2Model` stays almost identical—same embeddings (50,257 tokens and 1,024 positions to 768 dimensions), same dropout (10%), same 12 transformer blocks, and same LayerNorm layers. The overall layout doesn’t change much.
- **Attention Changes**: In the attention mechanism:
  - `c_attn` (originally a convolution from 768 to 2,304) is now wrapped in a `lora.Linear` layer. This keeps the original convolution (called the "base layer") but adds two tiny trainable layers: `lora_A` (768 to 8 dimensions) and `lora_B` (8 back to 2,304). These are low-rank matrices, meaning they’re much smaller than the original.
  - `c_proj` (originally 768 to 768) gets the same treatment: a `lora.Linear` wrapper with `lora_A` (768 to 8) and `lora_B` (8 to 768).
  - There’s also an extra `lora_dropout` (10%) alongside the regular attention dropouts, but the core dropout settings stay the same.
- **MLP Changes**: In the MLP:
  - `c_fc` (768 to 3,072) stays as a regular convolution with no LoRA applied.
  - `c_proj` (3,072 to 768) gets the LoRA treatment: wrapped in `lora.Linear` with `lora_A` (3,072 to 8) and `lora_B` (8 to 768), plus `lora_dropout`.
  - The GELU activation and regular dropout remain unchanged.
- **Score Layer**: The final linear layer (768 to 2) is now wrapped in a `ModulesToSaveWrapper`. This keeps the original layer but adds a mechanism to save specific parts during training, though the layer itself doesn’t change much.
- **Key Difference**: With LoRA, most of the original 124 million parameters are frozen. Instead, you only train the small LoRA layers (e.g., `lora_A` and `lora_B`), which might add just thousands of parameters instead of millions. This makes fine-tuning way more efficient.

---

### The Big Picture
- **Before PEFT**: You’ve got a full GPT-2 model where everything can be tweaked. It’s powerful but heavy—every weight gets updated, so you need lots of resources.
- **After PEFT**: You’re still using GPT-2, but LoRA sprinkles in these tiny, trainable layers on top of the attention and MLP projections. The bulk of the model stays frozen, and you only adjust these small additions. It’s like giving the model a lightweight upgrade instead of rebuilding the whole thing.

In short, the "After PEFT" version keeps the same GPT-2 soul but makes it much leaner and faster to fine-tune by focusing on a handful of new parameters rather than the whole beast!

In [ ]:
# the the trainable parameters
peft_model.print_trainable_parameters()

## Training the PEFT model

In [ ]:
# Let's train the model
# Train the model
# Let's create the trainer
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()

In [ ]:
# Let's evaluate the model
test_results = trainer.evaluate(tokenized_dataset['test'])
print("Test Results:", test_results)


In [ ]:
# Let's see the post-training score to ensure correct loading of model and it's weight during inference
post_training_score = peft_model.score.weight[0][:5].tolist()
print("Post-training score.weight:", post_training_score)

In [ ]:
import os
import json

# Path to the trainer_state.json from the latest checkpoint
checkpoint_path = "./gpt2-lora-rotten-tomatoes-classification/checkpoint-2134"

# Construct the full path to trainer_state.json
trainer_state_path = os.path.join(checkpoint_path, "trainer_state.json") # Join the directory with the file name

# Load the trainer state
with open(trainer_state_path, "r") as f:
    trainer_state = json.load(f)

In [ ]:
trainer_state

In [ ]:
trainer_state['log_history']

In [ ]:
import matplotlib.pyplot as plt
steps = []
train_loss = []
val_loss = []
step_train_loss = None

for entry in trainer_state["log_history"]:
    if "loss" in entry:
        step_train_loss = entry["loss"]
    if "eval_loss" in entry:
        steps.append(entry['step'])
        val_loss.append(entry["eval_loss"])
        train_loss.append(step_train_loss)

plt.plot(steps, train_loss, label="Train Loss", color="blue", marker='x')
plt.plot(steps, val_loss, label="Validation Loss", color="orange", marker='x')
plt.xlabel("Eval Steps")
plt.ylabel("Loss")
plt.title("Train vs Validation Loss")
plt.legend()
plt.grid(True)
plt.show()

### Saving the Model After Training
With current `Trainer` setup, we're already saving checkpoints during training (via `save_strategy="steps"` in `training_args`). However, since we’re using PEFT with LoRA, the default saving behavior only saves the LoRA adapter weights—not the full base model—because that’s the efficient part of PEFT. You have two options:

1. **Save Only the LoRA Adapters** (smaller footprint, requires base model to load).
2. **Save the Full Merged Model** (base model + LoRA weights combined, standalone).

I’ll cover both approaches.

#### Option 1: Save Only the LoRA Adapters
This is the default behavior with PEFT and aligns with your current setup.

```python
# Your training setup
from transformers import Trainer, DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model=peft_model,  # Your LoRA-adapted model
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Save the LoRA adapters and tokenizer to a specific directory
output_path = "/content/drive/MyDrive/NepGPT/PEFT/lora_adapters"
trainer.save_model(output_path)  # Saves LoRA weights and config
tokenizer.save_pretrained(output_path)  # Saves tokenizer
```

- **What’s Saved**:
  - `adapter_model.safetensors` (or `.bin`): The LoRA adapter weights.
  - `adapter_config.json`: The PEFT configuration (e.g., LoRA rank, target modules).
  - Tokenizer files (e.g., `tokenizer.json`, `vocab.txt`).
- **Size**: Very small (e.g., a few MB), since only the adapter weights are saved.
- **Note**: Your `training_args` already saves checkpoints to `output_dir`, so this is just an explicit final save. The best model (per `load_best_model_at_end=True`) will already be loaded into `peft_model` after `trainer.train()`.

#### Option 2: Save the Full Merged Model
If you want a standalone model for inference without needing to reload the base model separately, merge the LoRA adapters into the base model and save it.

```python
# After training
trainer.train()

# Merge the LoRA weights into the base model
merged_model = peft_model.merge_and_unload()  # Merges adapters into base model and unloads PEFT

# Save the full merged model and tokenizer
output_path = "/content/drive/MyDrive/NepGPT/PEFT/full_merged_model"
merged_model.save_pretrained(output_path)  # Saves full model weights and config
tokenizer.save_pretrained(output_path)  # Saves tokenizer
```

- **What’s Saved**:
  - `model.safetensors` (or `pytorch_model.bin`): The full model weights (base + merged LoRA).
  - `config.json`: Model configuration.
  - Tokenizer files.
- **Size**: Larger (e.g., hundreds of MB for GPT-2), since it includes the entire base model.
- **Benefit**: No need to handle PEFT separately during inference.

---

### Loading the Model for Inference
The loading process depends on which saving option you chose.

#### Loading Option 1: LoRA Adapters Only
You’ll need the base model (e.g., `gpt2`) and the saved LoRA adapters.

```python
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import PeftModel

# Load the base model
base_model_name = "gpt2"  # Replace with your actual base model name if different
base_model = AutoModelForSequenceClassification.from_pretrained(base_model_name)

# Load the tokenizer
adapter_path = "/content/drive/MyDrive/NepGPT/PEFT/lora_adapters"
tokenizer = AutoTokenizer.from_pretrained(adapter_path)

# Load the LoRA adapters and apply them to the base model
peft_model = PeftModel.from_pretrained(base_model, adapter_path)

# Set the model to evaluation mode
peft_model.eval()

# Example inference
inputs = tokenizer("This movie is great!", return_tensors="pt")
outputs = peft_model(**inputs)
logits = outputs.logits
prediction = logits.argmax(-1).item()
print("Prediction:", prediction)
```

- **Key Points**:
  - You must specify the original base model (`gpt2` or whatever you used).
  - The `PeftModel` combines the base model with the LoRA adapters dynamically.

#### Loading Option 2: Full Merged Model
If you saved the merged model, loading is simpler—no PEFT required.

```python
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the full merged model and tokenizer
model_path = "/content/drive/MyDrive/NepGPT/PEFT/full_merged_model"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Set the model to evaluation mode
model.eval()

# Example inference
inputs = tokenizer("This movie is great!", return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
prediction = logits.argmax(-1).item()
print("Prediction:", prediction)
```

- **Key Points**:
  - No PEFT dependency—just a standard Hugging Face model.
  - Ready to use out of the box.

---

### Recommendation
- **Use Option 1 (LoRA Adapters)** if:
  - You want to save disk space and share only the adapters.
  - You’re fine with loading the base model separately during inference.
- **Use Option 2 (Full Merged Model)** if:
  - You need a standalone model for deployment or simpler inference.
  - Disk space isn’t a concern.

Since I am using Google Drive and likely experimenting, I’d start with **Option 1** to keep things lightweight and aligned with PEFT’s philosophy. If you need a one-click inference solution later, switch to Option 2.



In [ ]:
# Option 1:
# Save the LoRA adapters and tokenizer to a specific directory
output_path = "./lora_adapters"
trainer.save_model(output_path)  # Saves LoRA weights and config
tokenizer.save_pretrained(output_path)  # Saves tokenizer

In [ ]:
# Option 2:
# Merge the LoRA weights into the base model
merged_model = peft_model.merge_and_unload()  # Merges adapters into base model and unloads PEFT

# Save the full merged model and tokenizer
output_path = "./full_merged_model"
merged_model.save_pretrained(output_path)  # Saves full model weights and config
tokenizer.save_pretrained(output_path)  # Saves tokenizer

## Inference Using Option 1: LoRA Adapters

You might get this warning:
`Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.`
What is this warning? GPT2 model is used for generating the text not for the classification. But we are using the `gpt2` for classification. So we have score layer in the below architecture which is randomly initialized that acts as the classification head.
```python
GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)
```
When we fin-tune this model using LoRA from the peft library we only choose to fine-tune the layers as shown in the config below:
```python
lora_config = LoraConfig(
    r=8,                    # Rank of the adaptation
    lora_alpha=32,          # Scaling factor
    target_modules=["c_attn", "c_proj"],  # layers to fine-tune in GPT-2
    lora_dropout=0.1,       # Dropout for regularization
    fan_in_fan_out=True,    # Match Conv1D expectation, eliminates the warning
    task_type="SEQ_CLS"   # For sequence Classification
)
```
Since `score.weight` isn’t present in the gpt2 checkpoint, Hugging Face initializes it randomly when you load `AutoModelForSequenceClassification`. This is what triggers the warning.

How to avoid this warning?
We can completely ignore this warning as it is seen just while loading the base model before it is fine-tuned and warning the user you might have to TRAIN this model.

Also we need to keep in mind that when we initialize the model with `peft_model = PeftModel.from_pretrained(base_model, lora_config)`, the score layer remains a regular Linear layer (not adapted by LoRA).
During training, the optimizer (controlled by Trainer) updates `score.weight` fully because unless not frozen or adapted by LoRA. We can check wether the it is optimized or not by
```python
# After initializing peft_model
print("Is score.weight trainable?", peft_model.score.weight.requires_grad)  # Should print True
```
Therefore, later when you load the adapter after fine-tuning with the base model the completely fine-tuned `score.weight` get's loaded. No need to worry about the warning at all.  

Also to be extra cautious we can check after training and after saving the model:
```python
# After training
trainer.train()
post_training_score = peft_model.score.weight[0][:5].tolist()
print("Post-training score.weight:", post_training_score)
peft_model.save_pretrained(output_path)
tokenizer.save_pretrained(output_path)

# Reload
base_model = AutoModelForSequenceClassification.from_pretrained("gpt2")
peft_model = PeftModel.from_pretrained(base_model, output_path)
loaded_score = peft_model.score.weight[0][:5].tolist()
print("Loaded score.weight:", loaded_score)
print("Match?", post_training_score == loaded_score)
```
If `Match? True`, warning is just noise, and model is working as intended.

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import PeftModel

# Load the base model
base_model_name = "gpt2"  # Replace with your actual base model name if different
base_model = AutoModelForSequenceClassification.from_pretrained(base_model_name)

In [ ]:
# Load the tokenizer
adapter_path = "./lora_adapters"
tokenizer = AutoTokenizer.from_pretrained(adapter_path)

with open(f"{adapter_path}/id2label.json", "r") as f:
    id2label = json.load(f)

# Load the LoRA adapters and apply them to the base model
peft_model = PeftModel.from_pretrained(base_model, adapter_path)

loaded_score = peft_model.score.weight[0][:5].tolist()
print("Loaded score.weight:", loaded_score)
print("Match?", post_training_score == loaded_score)

# Set the model to evaluation mode
peft_model.eval()

# Example inference
inputs = tokenizer("This movie is great!", return_tensors="pt")
outputs = peft_model(**inputs)
logits = outputs.logits
prediction_id = logits.argmax(-1).item()
# Convert prediction_id to string before accessing id2label
prediction_label = id2label[str(prediction_id)]
print("Input:", inputs)
print("Prediction:", prediction_label)

## Inference Using Option 2: Full-merged Model

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the full merged model and tokenizer
model_path = "./full_merged_model"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Set the model to evaluation mode
model.eval()

# Example inference
inputs = tokenizer("This movie is great!", return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
prediction_id = logits.argmax(-1).item()
# Convert prediction_id to string before accessing id2label
prediction_label = id2label[str(prediction_id)]
print("Input:", inputs)
print("Prediction:", prediction_label)